Importing modules

In [1]:
import pandas as pd
import json
import csv
from collections import defaultdict
import matplotlib.pyplot as plt
%matplotlib inline

import ast

In [2]:
raw_data = pd.read_csv('~/Documents/6242 Project/fall2018cse6242/data/arizona_business_details.csv', skipinitialspace=True)

In [33]:
raw_data['postal_code'] = raw_data['postal_code'].fillna(0)

raw_data['postal_code'] = raw_data['postal_code'].astype(int)


In [28]:
attributes = set()
for each in raw_data['attributes']:
    line = {}
    try:
        each = eval(each)
    except TypeError:
        continue
    for k in each.keys():
        attributes.add(k)

attributes = list(attributes)

In [45]:
def get_attributes(zipcode):

    attribute_count = defaultdict(int)
    data = raw_data[raw_data['postal_code'] == zipcode]
    n = data.shape[0]
    for each in data['attributes']:
        try:
            for attribute in attributes:
                if attribute in each:
                    attribute_count[attribute] += 1
        except TypeError:
            continue
    count_df = pd.DataFrame.from_dict(attribute_count, orient='index', columns= ['count'])
    count_df['percent_of_total'] = count_df['count']/n
    count_df = count_df.sort_values(by = 'percent_of_total', inplace=False, ascending=False)

    return count_df

Building the daframe for regression

In [60]:
features = ['BusinessAcceptsCreditCards',
 'RestaurantsTakeOut',
 'RestaurantsGoodForGroups',
 'GoodForKids',
 'RestaurantsDelivery',
 'RestaurantsReservations',
 'OutdoorSeating',
 'Alcohol',
 'RestaurantsTableService',
 'HasTV',
 'WiFi',
 'BikeParking',
 'Caters',
 'WheelchairAccessible']

check = ['False', 'False', 'False', 'False', 'False', 'False', 'False', 'none', 'False', 'False', 'no', 
         'False', 'False', 'False']

nest_features = ['Ambience', 'GoodForMeal', 'BusinessParking']
add_features = {}
add_features['Ambience'] = ['romantic', 'intimate', 'classy', 'hipster', 'divey', 'touristy', 'trendy', 'upscale', 'casual']
add_features['GoodForMeal'] = ['dessert', 'latenight', 'lunch', 'dinner', 'breakfast', 'brunch']
add_features['BusinessParking'] = ['garage', 'street', 'validated', 'lot', 'valet']



In [61]:
all_features = features + nest_features

In [62]:
def attribute_check(attribute, restaurant, check):
    if attribute in restaurant:
        if restaurant[attribute] == check:
            return 0
        else:
            return 1
    else:
        return 0

In [219]:
def create_dataframe(zipcode):
    attributes_zip = get_attributes(zipcode)
    attributes = attributes_zip[attributes_zip['percent_of_total'] > 0.45].index.values.tolist()

#     print (list(set(attributes) - set(all_features)))
    
    if 'RestaurantsPriceRange2' in attributes:
        attributes.remove('RestaurantsPriceRange2')
    data = raw_data[raw_data['postal_code'] == zipcode]
    dframe = pd.DataFrame()
    for index, row in data.iterrows():

        try:
            each = eval(row['attributes'])
            line = {}
            line['rating'] = row['stars']
#             line['zipcode'] = row['postal_code']
            for i in range(len(features)):
                line[features[i]] = attribute_check(features[i], each, check[i])

            # adding the nest features 
            for feature in nest_features:
                if feature in each: 
                    for cat in add_features[feature]:
                        if ast.literal_eval(each[feature])[cat] == True:
                            line[feature + '_' + cat] = 1
                        else:
                            line[feature + '_' + cat] = 0
                else:
                    for cat in add_features[feature]:
                            line[feature + '_' + cat] = 0
                    

            # Adding Noiselevel Data 
#             if 'NoiseLevel' in each:
#                 line['NoiseLevel'] = each['NoiseLevel']
#             else:
#                 line['NoiseLevel'] = 'missing'

    #         # Adding RestaurantsAttire data
    #         if 'RestaurantsAttire' in each:
    #             line['RestaurantsAttire'] = each['RestaurantsAttire']
    #         else:
    #             line['RestaurantsAttire'] = 'missing'


            # converting the dict as df 
            temp_df = pd.DataFrame([line])
            dframe = dframe.append(temp_df, ignore_index=True, sort=False)

        except TypeError:
            continue
    return dframe

In [223]:
create_dataframe(85002)



""


In [224]:
len(dframe)

0

Linear Regression Model Building

In [258]:
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns; sns.set()

from sklearn.model_selection import train_test_split


def zip_attributes(zipcode):
    attributes_list = {} 

    dframe = create_dataframe(zipcode)
    dframe = dframe.fillna(0)
    if len(dframe) > 0:
    
        train, test = train_test_split(dframe, test_size=0)

        X_train = train.loc[:, dframe.columns != 'rating']
        y_train = train['rating']

        X_test = test.loc[:, dframe.columns != 'rating']
        y_test = test['rating']

        # Create linear regression object
        regr = linear_model.LinearRegression()

        # Train the model using the training sets
        regr.fit(X_train, y_train)

        # Make predictions using the testing set
        # y_pred = regr.predict(X_test)

        # # The coefficients
        # print('Coefficients: \n', regr.coef_)
        # # The mean squared error
        # print("Mean squared error: %.2f"
        #       % mean_squared_error(y_test, y_pred))
        # # Explained variance score: 1 is perfect prediction
        # print('Variance score: %.2f' % r2_score(y_test, y_pred))

        # # Plot outputs
        # plt.plot(X_test, y_pred, color='blue', linewidth=3)

        # plt.xticks(())
        # plt.yticks(())

        # plt.show()

        attributes = list(X_train.columns)
        imp_attributes = defaultdict()
        for i in range(len(attributes)):
            if regr.coef_[i] >0:
                imp_attributes[attributes[i]] = regr.coef_[i]


        dic_attributes = dict(sorted(imp_attributes.items(), key=lambda kv: -kv[1])[:8])

        for d in dic_attributes:
            total = len(train)
            count = train[d].sum()
            if count > 0:
                attributes_list[d] = count/total
        attributes_list = dict(sorted(attributes_list.items(), key=lambda kv: -kv[1]))
#         attributes_list['zipcode'] = zipcode
    return attributes_list

In [259]:
# zipcodes = range(85001, 85005)

zipcodes = [85323,
 85007,
 85033,
 85023,
 85253,
 85257,
 85305,
 85258,
 85204,
 85338,
 85297,
 85044,
 85281,
 85001,
 85032,
 85331,
 85383,
 85224,
 85018,
 85306,
 85282,
 85286,
 85022,
 85210,
 85028,
 85250,
 85395,
 85296,
 85302,
 85048,
 85382,
 85207,
 85259,
 85008,
 85051,
 85202,
 85255,
 85212,
 85024,
 85283,
 85260,
 85003,
 85029,
 85351,
 85340,
 85251,
 85215,
 85019,
 85006,
 85201,
 85284,
 85034,
 85085,
 85020,
 85303,
 85295,
 85013,
 85225,
 85308,
 85256,
 85304,
 85233,
 85353,
 85203,
 85226,
 85326,
 85254,
 85054,
 85262,
 85266,
 85073,
 85307,
 85385,
 85009,
 85035,
 85043,
 85208,
 85015,
 85545,
 85381,
 85004,
 85361,
 85064,
 85031,
 85012,
 85016,
 85234,
 85248,
 85213,
 85017,
 85042,
 85037,
 85206,
 85268,
 85236,
 85378,
 85379,
 85086,
 85301,
 85374,
 85027,
 85249,
 85053,
 85205,
 85298,
 85050,
 85392,
 85375,
 85335,
 85388,
 85363,
 85209,
 85014,
 85287,
 85339,
 85377,
 85040,
 85143,
 85021,
 85041,
 85083,
 85345,
 85821,
 85310,
 85045,
 85058,
 85394,
 85373,
 85087,
 85005,
 85327,
 85318,
 85396,
 85329,
 85311,
 85355,
 85324,
 85264,
 85142,
 85010,
 85252,
 85140,
 85280,
 84295]

In [260]:
attributes_by_zipcode = {}
zipcode_attributes = {}
for zipcode in zipcodes:
    zipcode_attributes[zipcode] = zip_attributes(zipcode)


In [261]:
zipcode_attributes

{85323: {'BusinessAcceptsCreditCards': 0.9761904761904762,
  'GoodForKids': 0.8690476190476191,
  'BusinessParking_lot': 0.47619047619047616,
  'GoodForMeal_lunch': 0.4642857142857143,
  'Ambience_casual': 0.42857142857142855,
  'RestaurantsTableService': 0.38095238095238093,
  'HasTV': 0.36904761904761907,
  'BusinessParking_street': 0.03571428571428571},
 85007: {'RestaurantsGoodForGroups': 0.7272727272727273,
  'GoodForMeal_lunch': 0.4727272727272727,
  'Caters': 0.43636363636363634,
  'HasTV': 0.32727272727272727,
  'Ambience_casual': 0.32727272727272727,
  'BusinessParking_street': 0.21818181818181817,
  'BusinessParking_garage': 0.01818181818181818,
  'GoodForMeal_dessert': 0.01818181818181818},
 85033: {'RestaurantsTakeOut': 0.9682539682539683,
  'RestaurantsGoodForGroups': 0.8571428571428571,
  'HasTV': 0.36507936507936506,
  'GoodForMeal_lunch': 0.31746031746031744,
  'GoodForMeal_dinner': 0.14285714285714285,
  'OutdoorSeating': 0.14285714285714285,
  'GoodForMeal_brunch': 0.

In [262]:
df = pd.DataFrame.from_dict(zipcode_attributes)
df.to_csv('zipcode_attributes.csv')